In [ ]:
!pip install -q pyjwt

In [ ]:
import requests, jwt, os
from IPython.display import JSON

CATALOG_URL = "http://lakekeeper:8181/catalog"
MANAGEMENT_URL = "http://lakekeeper:8181/management"
KEYCLOAK_TOKEN_URL = "http://keycloak:8080/realms/iceberg/protocol/openid-connect/token"

# Get host IP from environment variable (set by lakekeeper.sh)
HOST_IP = os.environ.get('HOST_IP', '192.168.178.88')
print(f"Using MinIO endpoint: http://{HOST_IP}:9000")

# Bootstrapping Lakekeeper
This Notebook performs bootstrapping via python requests. It only works if the server hasn't previously bootstrapped using the UI!

## 1. Sign in
First, we need to obtain a token from our Identity Provider. In this example a `Keycloak` is running beside Lakekeeper. A few users have been pre-created in Keycloak for this example. We are now logging into Keycloak as the technical user (client) `trino`. If a human user bootstraps the catalog, we recommend to use the UI.

In [ ]:
# Login to Keycloak
CLIENT_ID = "trino"
CLIENT_SECRET = "AK48QgaKsqdEpP9PomRJw7l2T7qWGHdZ"

response = requests.post(
    url=KEYCLOAK_TOKEN_URL,
    data={
        "grant_type": "client_credentials",
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "scope": "lakekeeper"
    },
    headers={"Content-type": "application/x-www-form-urlencoded"},
)
response.raise_for_status()
access_token = response.json()['access_token']

# Lets inspect the token we got to see that our application name is available:
JSON(jwt.decode(access_token, options={"verify_signature": False}))

Now that we have the access token, we can query the server info Endpoint. 
On first launch it will show bootstrapped `'bootstrapped': false`.
The full API documentation is available as part of the Repository and hosted by Lakekeeper: http://localhost:8181/swagger-ui/#/

In [ ]:
response = requests.get(
    url=f"{MANAGEMENT_URL}/v1/info",
    headers={"Authorization": f"Bearer {access_token}"},
)
response.raise_for_status()
JSON(response.json())
# On first launch it shows "bootstrapped": False

## 2. Bootstrap

In [ ]:
response = requests.post(
    url=f"{MANAGEMENT_URL}/v1/bootstrap",
    headers={
        "Authorization": f"Bearer {access_token}"
    },
    json={
        "accept-terms-of-use": True,
    },
)
response.raise_for_status()

## 3. Register Peter User

Peter exists in Keycloak and has been pre-configured in the Cedar policy files. We'll now register him in Lakekeeper programmatically.

In [ ]:
# Create Peter user in Lakekeeper
# Peter's ID from Keycloak (you can get this from the token or Keycloak admin)
# For this example, we'll use a standard format: oidc~{keycloak-user-id}
# We need to get Peter's Keycloak user ID first

response = requests.post(
    url=f"{MANAGEMENT_URL}/v1/user",
    headers={
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    },
    json={
        "user-id": "peter@iceberg.local"
    }
)
if response.status_code == 409:
    print("Peter already exists")
else:
    response.raise_for_status()
    print("Peter created successfully!")
    JSON(response.json())

You can now refresh the UI page and should see the default Lakehouse.

**Note**: With Cedar authorization, Peter's permissions are managed via the policy file at `/home/jovyan/policies/policies.cedar`.

## 4. Creating a Warehouse

In [ ]:
response = requests.post(
    url=f"{MANAGEMENT_URL}/v1/warehouse",
    headers={
        "Authorization": f"Bearer {access_token}"
    },
    json={
      "warehouse-name": "demo",
      "storage-profile": {
        "type": "s3",
        "bucket": "examples",
        "key-prefix": "initial-warehouse",
        "endpoint": f"http://{HOST_IP}:9000",
        "region": "local-01",
        "path-style-access": True,
        "flavor": "s3-compat",
        "sts-enabled": True
      },
      "storage-credential": {
        "type": "s3",
        "credential-type": "access-key",
        "aws-access-key-id": "minio-root-user",
        "aws-secret-access-key": "minio-root-password"
      }
    }
)

if response.status_code not in [200, 201]:
    print(f"Error: {response.status_code}")
    print(response.text)
    
response.raise_for_status()
JSON(response.json())
